In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

# Importing libraries
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import Dense, Dropout, Activation

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


Centralized CNN (merge all subjects' data in a pool)

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Dropout, Flatten, LSTM, Dense, InputLayer
from sklearn.metrics import accuracy_score

# ---------------------------
# Config
# ---------------------------
dataset_path = "/content/drive/MyDrive/UCI_HAR_Dataset"
batch_size = 32
epochs_general = 30   # epochs for general model
epochs_finetune = 5   # epochs for per-subject fine-tuning
n_steps, n_length = 4, 32
n_hidden = 16

SIGNALS = [
    "body_acc_x", "body_acc_y", "body_acc_z",
    "body_gyro_x", "body_gyro_y", "body_gyro_z",
    "total_acc_x", "total_acc_y", "total_acc_z"
]

from sklearn.model_selection import train_test_split

def load_data(dataset_path, test_size=0.2, seed=42):
    def _read_csv(filename):
        return pd.read_csv(filename, delim_whitespace=True, header=None)

    def load_signals(subset):
        signals_data = []
        for signal in SIGNALS:
            filename = f"{dataset_path}/{subset}/Inertial Signals/{signal}_{subset}.txt"
            signals_data.append(_read_csv(filename).to_numpy())
        return np.transpose(signals_data, (1, 2, 0))  # (samples, timesteps=128, 9 signals)

    def load_y(subset):
        filename = f"{dataset_path}/{subset}/y_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    def load_subjects(subset):
        filename = f"{dataset_path}/{subset}/subject_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    # Load full dataset (train + test)
    X_train, y_train, subj_train = load_signals("train"), load_y("train"), load_subjects("train")
    X_test,  y_test,  subj_test  = load_signals("test"),  load_y("test"),  load_subjects("test")

    # Merge all
    X_all = np.vstack([X_train, X_test])
    y_all = np.concatenate([y_train, y_test])
    subjects = np.concatenate([subj_train, subj_test])  # not used anymore

    print(f"✅ Loaded all subjects together: {X_all.shape[0]} samples")

    # Shuffle + split (ignore subject IDs)
    train_X, test_X, train_y, test_y = train_test_split(
        X_all, y_all, test_size=test_size, random_state=seed, shuffle=True, stratify=y_all
    )

    print(f"📊 Final split -> Train: {train_X.shape}, Test: {test_X.shape}")

    return train_X, train_y, test_X, test_y


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder

# ---------------------------
# Load dataset (merged subjects, 80/20 split)
# ---------------------------
X_train, y_train, X_test, y_test = load_data(dataset_path, test_size=0.2, seed=42)

# ---------------------------
# Encode labels
# ---------------------------
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded  = le.transform(y_test)

n_classes = len(le.classes_)
print("Classes found:", le.classes_)

# reshape into subsequences for CNN-GRU
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, X_train.shape[2]))
X_test  = X_test.reshape((X_test.shape[0],  n_steps, n_length, X_test.shape[2]))

# convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test_encoded, dtype=torch.long)

# datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)



# ---------------------------
# Define pure CNN model
# ---------------------------
class CNN1D(nn.Module):
    def __init__(self, n_steps, n_length, input_dim, n_classes):
        super(CNN1D, self).__init__()

        # Conv layers
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=5, padding=2)
        self.bn1   = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.bn2   = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm1d(256)

        self.dropout = nn.Dropout(0.5)
        self.pool    = nn.AdaptiveMaxPool1d(1)  # global pooling

        # FC layer
        self.fc = nn.Linear(256, n_classes)

    def forward(self, x):
        # x: (batch, n_steps, n_length, input_dim)
        # Merge steps and time
        x = x.reshape(x.size(0), x.size(1) * x.size(2), x.size(3))  # (B, steps*len, channels)
        x = x.permute(0, 2, 1)  # (B, channels=input_dim, seq_len)

        # Conv stack
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)

        # Global pooling
        x = self.pool(x).squeeze(-1)  # (B, 256)

        # Classifier
        logits = self.fc(x)
        return logits




# ---------------------------
# Train general model
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
general_model = CNN1D(n_steps, n_length, X_train.shape[3], n_classes).to(device)


# Label smoothing loss
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.Adam(general_model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_val_loss = float("inf")
patience_counter = 0
early_stop_patience = 10

for epoch in range(epochs_general):
    # --- Train ---
    general_model.train()
    total_loss, correct, total = 0, 0, 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = general_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(general_model.parameters(), max_norm=5.0)
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
    train_loss = total_loss / len(train_loader)
    train_acc = correct / total

    # --- Validation ---
    general_model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = general_model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    val_loss = val_loss / len(test_loader)
    val_acc = correct / total

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{epochs_general} "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_weights = general_model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print("Early stopping triggered!")
            break


# restore best weights
general_model.load_state_dict(best_weights)

# Save weights of general model
general_weights = general_model.state_dict()


✅ Loaded all subjects together: 10299 samples
📊 Final split -> Train: (8239, 128, 9), Test: (2060, 128, 9)
Classes found: [1 2 3 4 5 6]
Epoch 1/30 Train Loss: 0.6289, Train Acc: 0.9183 | Val Loss: 0.6986, Val Acc: 0.9417
Epoch 2/30 Train Loss: 0.5304, Train Acc: 0.9539 | Val Loss: 0.6722, Val Acc: 0.9684
Epoch 3/30 Train Loss: 0.5103, Train Acc: 0.9640 | Val Loss: 0.6657, Val Acc: 0.9680
Epoch 4/30 Train Loss: 0.5023, Train Acc: 0.9641 | Val Loss: 0.6806, Val Acc: 0.9709
Epoch 5/30 Train Loss: 0.4975, Train Acc: 0.9664 | Val Loss: 0.6956, Val Acc: 0.9748
Epoch 6/30 Train Loss: 0.4935, Train Acc: 0.9692 | Val Loss: 0.6748, Val Acc: 0.9743
Epoch 7/30 Train Loss: 0.4885, Train Acc: 0.9722 | Val Loss: 0.6785, Val Acc: 0.9801
Epoch 8/30 Train Loss: 0.4784, Train Acc: 0.9769 | Val Loss: 0.6674, Val Acc: 0.9796
Epoch 9/30 Train Loss: 0.4783, Train Acc: 0.9762 | Val Loss: 0.6640, Val Acc: 0.9825
Epoch 10/30 Train Loss: 0.4770, Train Acc: 0.9763 | Val Loss: 0.6732, Val Acc: 0.9820
Epoch 11/30 T

In [ ]:
def load_data_by_subject(dataset_path):
    def _read_csv(filename):
        return pd.read_csv(filename, delim_whitespace=True, header=None)

    def load_signals(subset):
        signals_data = []
        for signal in SIGNALS:
            filename = f"{dataset_path}/{subset}/Inertial Signals/{signal}_{subset}.txt"
            signals_data.append(_read_csv(filename).to_numpy())
        return np.transpose(signals_data, (1, 2, 0))  # (samples, timesteps=128, 9 signals)

    def load_y(subset):
        filename = f"{dataset_path}/{subset}/y_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    def load_subjects(subset):
        filename = f"{dataset_path}/{subset}/subject_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    # Load train & test partitions
    X_train, y_train, subj_train = load_signals("train"), load_y("train"), load_subjects("train")
    X_test,  y_test,  subj_test  = load_signals("test"),  load_y("test"),  load_subjects("test")

    # Merge everything
    X_all = np.vstack([X_train, X_test])
    y_all = np.concatenate([y_train, y_test])
    subjects = np.concatenate([subj_train, subj_test])

    # Organize by subject
    subject_dict = {}
    for subj_id in np.unique(subjects):
        mask = (subjects == subj_id)
        subject_dict[subj_id] = (X_all[mask], y_all[mask])

    print(f"✅ Loaded dataset: {X_all.shape[0]} samples from {len(subject_dict)} subjects")
    return subject_dict

In [ ]:
# Load per-subject data
subject_data = load_data_by_subject(dataset_path)

# Run inference for one subject (e.g., subject 5)
X_subj, y_subj = subject_data[5]
print("Subject 5 ->", X_subj.shape, y_subj.shape)

# Later: use your trained model
# preds = model.predict(X_subj)


✅ Loaded dataset: 10299 samples from 30 subjects
Subject 5 -> (302, 128, 9) (302,)


In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score

def evaluate_per_subject(general_model, subject_data, le, n_steps, n_length, device="cuda"):
    general_model.eval()
    subject_accuracies = {}

    with torch.no_grad():
        for subj_id, (X_subj, y_subj) in subject_data.items():
            # reshape into subsequences like training
            X_subj = X_subj.reshape((X_subj.shape[0], n_steps, n_length, X_subj.shape[2]))

            # encode labels with same LabelEncoder
            y_subj_encoded = le.transform(y_subj)

            # convert to torch tensors
            X_tensor = torch.tensor(X_subj, dtype=torch.float32).to(device)
            y_tensor = torch.tensor(y_subj_encoded, dtype=torch.long).to(device)

            # forward pass
            outputs = general_model(X_tensor)
            _, preds = torch.max(outputs, 1)

            acc = accuracy_score(y_tensor.cpu().numpy(), preds.cpu().numpy())
            subject_accuracies[subj_id] = acc

    # Convert to numpy array for stats
    acc_values = np.array(list(subject_accuracies.values()))
    mean_acc = acc_values.mean()
    std_acc = acc_values.std()

    print("\n📊 Per-subject accuracies:")
    for subj_id, acc in subject_accuracies.items():
        print(f"  Subject {subj_id}: {acc:.4f}")

    print(f"\n✅ Mean Accuracy: {mean_acc:.4f}, Std: {std_acc:.4f}")

    return subject_accuracies, mean_acc, std_acc


In [ ]:
# 1. Load subject-wise data
subject_data = load_data_by_subject(dataset_path)

# 2. Evaluate general_model on each subject
subject_accuracies, mean_acc, std_acc = evaluate_per_subject(
    general_model, subject_data, le, n_steps, n_length, device=device
)


✅ Loaded dataset: 10299 samples from 30 subjects

📊 Per-subject accuracies:
  Subject 1: 1.0000
  Subject 2: 0.9636
  Subject 3: 1.0000
  Subject 4: 0.9937
  Subject 5: 0.9834
  Subject 6: 0.9969
  Subject 7: 0.9870
  Subject 8: 0.9537
  Subject 9: 0.9861
  Subject 10: 0.9762
  Subject 11: 1.0000
  Subject 12: 1.0000
  Subject 13: 1.0000
  Subject 14: 1.0000
  Subject 15: 1.0000
  Subject 16: 0.9781
  Subject 17: 0.9946
  Subject 18: 1.0000
  Subject 19: 1.0000
  Subject 20: 1.0000
  Subject 21: 1.0000
  Subject 22: 1.0000
  Subject 23: 0.9973
  Subject 24: 1.0000
  Subject 25: 0.9756
  Subject 26: 1.0000
  Subject 27: 1.0000
  Subject 28: 0.9764
  Subject 29: 0.9942
  Subject 30: 1.0000

✅ Mean Accuracy: 0.9919, Std: 0.0122


# Domain Generalization

In [16]:

# ---------------------------
# Config
# ---------------------------
dataset_path = "/content/drive/MyDrive/UCI_HAR_Dataset"
test_subjects = [2, 5, 7, 8, 9, 10]  # hold-out subjects for testing
batch_size = 32
epochs_general = 10   # epochs for general model
epochs_finetune = 5   # epochs for per-subject fine-tuning
n_steps, n_length = 4, 32
n_hidden = 16

SIGNALS = [
    "body_acc_x", "body_acc_y", "body_acc_z",
    "body_gyro_x", "body_gyro_y", "body_gyro_z",
    "total_acc_x", "total_acc_y", "total_acc_z"
]

def load_data(dataset_path, test_subjects):
    def _read_csv(filename):
        return pd.read_csv(filename, delim_whitespace=True, header=None)

    def load_signals(subset):
        signals_data = []
        for signal in SIGNALS:
            filename = f"{dataset_path}/{subset}/Inertial Signals/{signal}_{subset}.txt"
            signals_data.append(_read_csv(filename).to_numpy())
        return np.transpose(signals_data, (1, 2, 0))  # (samples, timesteps=128, 9 signals)

    def load_y(subset):
        filename = f"{dataset_path}/{subset}/y_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    def load_subjects(subset):
        filename = f"{dataset_path}/{subset}/subject_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    X_train, y_train, subj_train = load_signals("train"), load_y("train"), load_subjects("train")
    X_test,  y_test,  subj_test  = load_signals("test"),  load_y("test"),  load_subjects("test")

    X_all = np.vstack([X_train, X_test])
    y_all = np.concatenate([y_train, y_test])
    subjects = np.concatenate([subj_train, subj_test])

    # Split by subject IDs
    train_mask = ~np.isin(subjects, test_subjects)
    test_mask  = np.isin(subjects, test_subjects)

    train_X, train_y = X_all[train_mask], y_all[train_mask]
    test_X,  test_y  = X_all[test_mask],  y_all[test_mask]
    train_subjects   = subjects[train_mask]

    print(f"Selected test subjects: {test_subjects}")
    print("Train:", train_X.shape, train_y.shape)
    print("Test :", test_X.shape, test_y.shape)

    return train_X, train_y, test_X, test_y, train_subjects

X_train, y_train, X_test, y_test, train_subjects = load_data(dataset_path, test_subjects=test_subjects)

# ---------------------------
# Encode labels
# ---------------------------
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded  = le.transform(y_test)

n_classes = len(le.classes_)
print("Classes found:", le.classes_)

# reshape into subsequences for CNN-GRU
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, X_train.shape[2]))
X_test  = X_test.reshape((X_test.shape[0],  n_steps, n_length, X_test.shape[2]))

# convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test_encoded, dtype=torch.long)

# datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

Selected test subjects: [2, 5, 7, 8, 9, 10]
Train: (8524, 128, 9) (8524,)
Test : (1775, 128, 9) (1775,)
Classes found: [1 2 3 4 5 6]


**CNN**

In [17]:
# ---------------------------
# Define pure CNN model
# ---------------------------
class CNN1D(nn.Module):
    def __init__(self, n_steps, n_length, input_dim, n_classes):
        super(CNN1D, self).__init__()

        # Conv layers
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=5, padding=2)
        self.bn1   = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.bn2   = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm1d(256)

        self.dropout = nn.Dropout(0.5)
        self.pool    = nn.AdaptiveMaxPool1d(1)  # global pooling

        # FC layer
        self.fc = nn.Linear(256, n_classes)

    def forward(self, x):
        # x: (batch, n_steps, n_length, input_dim)
        # Merge steps and time
        x = x.reshape(x.size(0), x.size(1) * x.size(2), x.size(3))  # (B, steps*len, channels)
        x = x.permute(0, 2, 1)  # (B, channels=input_dim, seq_len)

        # Conv stack
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)

        # Global pooling
        x = self.pool(x).squeeze(-1)  # (B, 256)

        # Classifier
        logits = self.fc(x)
        return logits




# ---------------------------
# Train general model
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
general_model = CNN1D(n_steps, n_length, X_train.shape[3], n_classes).to(device)


# Label smoothing loss
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.Adam(general_model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_val_loss = float("inf")
patience_counter = 0
early_stop_patience = 10

for epoch in range(epochs_general):
    # --- Train ---
    general_model.train()
    total_loss, correct, total = 0, 0, 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = general_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(general_model.parameters(), max_norm=5.0)
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
    train_loss = total_loss / len(train_loader)
    train_acc = correct / total

    # --- Validation ---
    general_model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = general_model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    val_loss = val_loss / len(test_loader)
    val_acc = correct / total

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{epochs_general} "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_weights = general_model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print("Early stopping triggered!")
            break


# restore best weights
general_model.load_state_dict(best_weights)

# Save weights of general model
general_weights = general_model.state_dict()

Epoch 1/10 Train Loss: 0.6000, Train Acc: 0.9352 | Val Loss: 0.8360, Val Acc: 0.8518
Epoch 2/10 Train Loss: 0.5110, Train Acc: 0.9673 | Val Loss: 0.8061, Val Acc: 0.8620
Epoch 3/10 Train Loss: 0.4952, Train Acc: 0.9716 | Val Loss: 0.8131, Val Acc: 0.8879
Epoch 4/10 Train Loss: 0.4866, Train Acc: 0.9750 | Val Loss: 0.8392, Val Acc: 0.8885
Epoch 5/10 Train Loss: 0.4851, Train Acc: 0.9749 | Val Loss: 0.8031, Val Acc: 0.8907
Epoch 6/10 Train Loss: 0.4811, Train Acc: 0.9767 | Val Loss: 0.8319, Val Acc: 0.8834
Epoch 7/10 Train Loss: 0.4736, Train Acc: 0.9803 | Val Loss: 0.8035, Val Acc: 0.9076
Epoch 8/10 Train Loss: 0.4741, Train Acc: 0.9802 | Val Loss: 0.7891, Val Acc: 0.9054
Epoch 9/10 Train Loss: 0.4705, Train Acc: 0.9817 | Val Loss: 0.8295, Val Acc: 0.9087
Epoch 10/10 Train Loss: 0.4690, Train Acc: 0.9826 | Val Loss: 0.8205, Val Acc: 0.8885


**CNN + Gradient Reversal Layer**

In [29]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

# ---------------------------
# Config
# ---------------------------
dataset_path = "/content/drive/MyDrive/UCI_HAR_Dataset"
test_subjects = [2, 5, 7, 8, 9, 10]  # hold-out subjects for testing
batch_size = 32
epochs_general = 10
n_steps, n_length = 4, 32

SIGNALS = [
    "body_acc_x", "body_acc_y", "body_acc_z",
    "body_gyro_x", "body_gyro_y", "body_gyro_z",
    "total_acc_x", "total_acc_y", "total_acc_z"
]

# ---------------------------
# Load data: returns subject ids per sample (train + test)
# ---------------------------
def load_data(dataset_path, test_subjects):
    def _read_csv(filename):
        return pd.read_csv(filename, delim_whitespace=True, header=None)

    def load_signals(subset):
        signals_data = []
        for signal in SIGNALS:
            filename = f"{dataset_path}/{subset}/Inertial Signals/{signal}_{subset}.txt"
            signals_data.append(_read_csv(filename).to_numpy())
        return np.transpose(signals_data, (1, 2, 0))  # (samples, timesteps, channels)

    def load_y(subset):
        filename = f"{dataset_path}/{subset}/y_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    def load_subjects(subset):
        filename = f"{dataset_path}/{subset}/subject_{subset}.txt"
        return _read_csv(filename)[0].to_numpy()

    # Load partitions
    X_train_part, y_train_part, subj_train_part = load_signals("train"), load_y("train"), load_subjects("train")
    X_test_part,  y_test_part,  subj_test_part  = load_signals("test"),  load_y("test"),  load_subjects("test")

    # Merge
    X_all = np.vstack([X_train_part, X_test_part])
    y_all = np.concatenate([y_train_part, y_test_part])
    subjects_all = np.concatenate([subj_train_part, subj_test_part])

    # Split by subject IDs (hold-out subjects -> test)
    train_mask = ~np.isin(subjects_all, test_subjects)
    test_mask  = np.isin(subjects_all, test_subjects)

    train_X, train_y, train_subj = X_all[train_mask], y_all[train_mask], subjects_all[train_mask]
    test_X,  test_y,  test_subj  = X_all[test_mask],  y_all[test_mask],  subjects_all[test_mask]

    print(f"Selected test subjects: {test_subjects}")
    print("Train samples:", train_X.shape[0], "Test samples:", test_X.shape[0])

    return train_X, train_y, train_subj, test_X, test_y, test_subj

# ---------------------------
# Load + preprocess for CNN + GRL
# ---------------------------
X_train, y_train, train_subj, X_test, y_test, test_subj = load_data(dataset_path, test_subjects=test_subjects)

# Encode activity labels
le_activity = LabelEncoder()
y_train_encoded = le_activity.fit_transform(y_train)
y_test_encoded  = le_activity.transform(y_test)
n_classes = len(le_activity.classes_)
print("Activity classes found:", le_activity.classes_)

# Encode training subjects only for GRL
le_subjects = LabelEncoder()
train_subj_encoded = le_subjects.fit_transform(train_subj)  # only training subjects
n_subjects = len(le_subjects.classes_)
print("Unique subjects in training:", n_subjects)

# Optionally encode test subjects (for analysis only, not used in GRL)
# For evaluation, GRL head is ignored
test_subj_encoded = np.array([-1]*len(test_subj))  # dummy labels for test

# Reshape for CNN input: (samples, n_steps, n_length, channels)
expected_timesteps = n_steps * n_length
assert X_train.shape[1] == expected_timesteps
assert X_test.shape[1]  == expected_timesteps

X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, X_train.shape[2]))
X_test  = X_test.reshape((X_test.shape[0],  n_steps, n_length, X_test.shape[2]))

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test_encoded, dtype=torch.long)
subj_train_tensor = torch.tensor(train_subj_encoded, dtype=torch.long)
# Test loader will ignore subject labels
subj_test_tensor  = torch.tensor(test_subj_encoded, dtype=torch.long)

# Create datasets
# Training dataset includes subject labels for GRL
train_dataset = TensorDataset(X_train_tensor, y_train_tensor, subj_train_tensor)
# Test dataset ignores subject labels to prevent IndexError
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Data loaders ready. Train batches:", len(train_loader), "Test batches:", len(test_loader))



Selected test subjects: [2, 5, 7, 8, 9, 10]
Train samples: 8524 Test samples: 1775
Activity classes found: [1 2 3 4 5 6]
Unique subjects in training: 24
Data loaders ready. Train batches: 267 Test batches: 56


In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import numpy as np
import time
import os

# ---------------------------
# GRL (Gradient Reversal Layer)
# ---------------------------
class GradReverse(Function):
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambda_ * grad_output, None

def grad_reverse(x, lambda_=1.0):
    return GradReverse.apply(x, lambda_)

# ---------------------------
# Subject-Invariant CNN (uses same CNN architecture as before)
# ---------------------------
class SubjectInvariantCNN(nn.Module):
    def __init__(self, n_steps, n_length, input_dim, n_classes, n_subjects, hidden_dim=128):
        super().__init__()
        self.n_steps = n_steps
        self.n_length = n_length
        self.input_dim = input_dim
        self.n_classes = n_classes
        self.n_subjects = n_subjects

        # Conv backbone (same as your CNN1D)
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=5, padding=2)
        self.bn1   = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.bn2   = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm1d(256)

        self.dropout = nn.Dropout(0.5)
        self.pool    = nn.AdaptiveMaxPool1d(1)  # global pooling

        # projection to hidden_dim used by both heads
        self.feature_proj = nn.Linear(256, hidden_dim)

        # Activity classifier
        self.fc_activity = nn.Linear(hidden_dim, n_classes)

        # Subject classifier (adversarial head)
        self.fc_subject = nn.Linear(hidden_dim, n_subjects)

    def forward(self, x, lambda_adv=1.0):
        """
        x: (batch, n_steps, n_length, channels)
        returns: logits_activity, logits_subject
        """
        # Merge steps and length into time axis, then conv expects (B, channels, seq_len)
        x = x.reshape(x.size(0), x.size(1) * x.size(2), x.size(3))   # (B, steps*len, channels)
        x = x.permute(0, 2, 1)  # (B, channels, seq_len)

        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(x).squeeze(-1)  # (B, 256)

        features = self.feature_proj(x)  # (B, hidden_dim)

        logits_activity = self.fc_activity(features)

        # apply GRL for adversarial subject classification
        rev_features = grad_reverse(features, lambda_adv)
        logits_subject = self.fc_subject(rev_features)

        return logits_activity, logits_subject

# ---------------------------
# Training function (fixed lambda, early stopping based on val accuracy)
# ---------------------------
def train_cnn_grl(
    train_loader,
    test_loader,
    n_steps,
    n_length,
    input_dim,
    n_classes,
    n_subjects,
    device=None,
    epochs=10,
    lr=5e-4,
    lambda_adv=1.0,  # fixed lambda
    weight_decay=1e-4,
    early_stop_patience=10,
    save_path="best_grl_model.pth"
):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    model = SubjectInvariantCNN(n_steps, n_length, input_dim, n_classes, n_subjects).to(device)

    criterion_activity = nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion_subject  = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_val_acc = 0.0  # track best accuracy
    patience = 0

    for epoch in range(1, epochs + 1):
        epoch_start = time.time()
        model.train()
        train_losses = []
        train_act_losses = []
        train_subj_losses = []
        train_correct = 0
        train_total = 0

        for X_batch, y_activity, y_subject in train_loader:
            X_batch, y_activity, y_subject = X_batch.to(device), y_activity.to(device), y_subject.to(device)

            optimizer.zero_grad()
            logits_act, logits_subj = model(X_batch, lambda_adv=lambda_adv)

            loss_act = criterion_activity(logits_act, y_activity)
            loss_sub = criterion_subject(logits_subj, y_subject)
            loss = loss_act - lambda_adv * loss_sub

            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            train_act_losses.append(loss_act.item())
            train_subj_losses.append(loss_sub.item())
            train_correct += (logits_act.argmax(dim=1) == y_activity).sum().item()
            train_total += y_activity.size(0)

        train_loss = float(np.mean(train_losses))
        train_act_loss = float(np.mean(train_act_losses))
        train_subj_loss = float(np.mean(train_subj_losses))
        train_acc = train_correct / train_total

        # ---------------------------
        # Validation (no GRL)
        # ---------------------------
        model.eval()
        val_losses = []
        val_act_losses = []
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for X_batch, y_activity in test_loader:  # only 2 items in test_loader
                X_batch, y_activity = X_batch.to(device), y_activity.to(device)
                logits_act, _ = model(X_batch, lambda_adv=0.0)
                loss_act = criterion_activity(logits_act, y_activity)

                val_losses.append(loss_act.item())
                val_act_losses.append(loss_act.item())
                val_correct += (logits_act.argmax(dim=1) == y_activity).sum().item()
                val_total += y_activity.size(0)

        val_loss = float(np.mean(val_losses))
        val_act_loss = float(np.mean(val_act_losses))
        val_acc = val_correct / val_total

        epoch_time = time.time() - epoch_start
        print(f"Epoch {epoch:03d} | {epoch_time:.1f}s | lambda {lambda_adv:.3f} | "
              f"Train loss {train_loss:.4f} (act {train_act_loss:.4f}, subj {train_subj_loss:.4f}), acc {train_acc:.4f} | "
              f"Val loss {val_loss:.4f}, acc {val_acc:.4f}")

        # Early stopping + save best based on **val accuracy**
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience = 0
            torch.save({
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": epoch,
                "val_acc": val_acc,
            }, save_path)
            print(f"  -> Saved best model to {save_path} (val_acc={val_acc:.4f})")
        else:
            patience += 1
            if patience >= early_stop_patience:
                print("Early stopping triggered.")
                break

    # Load best model weights
    if os.path.exists(save_path):
        ckpt = torch.load(save_path, map_location=device)
        model.load_state_dict(ckpt["model_state_dict"])
        print(f"Loaded best model from epoch {ckpt.get('epoch')} with val_acc {ckpt.get('val_acc'):.4f}")

    return model







In [39]:
# ---------------------------
# Setup and training script for CNN+GRL
# ---------------------------
import torch

# ---------------------------
# Dataset properties from loaded data
# ---------------------------
input_dim = X_train.shape[3]    # number of sensor channels (e.g., 9)
print("Input dim:", input_dim)

n_classes = len(le_activity.classes_)  # activity classes
print("Activity classes:", n_classes)

n_subjects = len(np.unique(train_subj))  # unique training subjects only
print("Subjects in training:", n_subjects)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------------
# Train CNN+GRL
# ---------------------------
trained_model = train_cnn_grl(
    train_loader=train_loader,
    test_loader=test_loader,
    n_steps=n_steps,
    n_length=n_length,
    input_dim=input_dim,
    n_classes=n_classes,
    n_subjects=n_subjects,
    device=device,
    epochs=30,
    lr=5e-4,
    lambda_adv=0,
    weight_decay=1e-4,
    early_stop_patience=10,
    save_path="best_grl_model.pth"
)

# ---------------------------
# Evaluate final model on test set
# ---------------------------
trained_model.eval()
test_correct, test_total = 0, 0

with torch.no_grad():
    for X_batch, y_act in test_loader:   # <- only 2 values
        X_batch = X_batch.to(device)
        y_act   = y_act.to(device)

        logits_act, _ = trained_model(X_batch, lambda_adv=0.0)  # no adversarial effect in inference
        preds = torch.argmax(logits_act, dim=1)

        test_correct += (preds == y_act).sum().item()
        test_total += y_act.size(0)

test_acc = test_correct / test_total
print(f"✅ Final Test Accuracy (Activity): {test_acc:.4f}")



Input dim: 9
Activity classes: 6
Subjects in training: 24
Epoch 001 | 33.7s | lambda 0.000 | Train loss 0.6076 (act 0.6076, subj 3.2812), acc 0.9291 | Val loss 0.8185, acc 0.8777
  -> Saved best model to best_grl_model.pth (val_acc=0.8777)
Epoch 002 | 35.2s | lambda 0.000 | Train loss 0.5163 (act 0.5163, subj 3.1781), acc 0.9636 | Val loss 0.8483, acc 0.8456
Epoch 003 | 33.9s | lambda 0.000 | Train loss 0.5127 (act 0.5127, subj 3.1781), acc 0.9640 | Val loss 0.8004, acc 0.8879
  -> Saved best model to best_grl_model.pth (val_acc=0.8879)
Epoch 004 | 35.2s | lambda 0.000 | Train loss 0.4967 (act 0.4967, subj 3.1781), acc 0.9708 | Val loss 0.8135, acc 0.8992
  -> Saved best model to best_grl_model.pth (val_acc=0.8992)
Epoch 005 | 34.1s | lambda 0.000 | Train loss 0.4913 (act 0.4913, subj 3.1781), acc 0.9713 | Val loss 0.8555, acc 0.8648
Epoch 006 | 35.4s | lambda 0.000 | Train loss 0.4853 (act 0.4853, subj 3.1781), acc 0.9761 | Val loss 0.8141, acc 0.8704
Epoch 007 | 34.0s | lambda 0.000 